In [3]:
import mxnet as mx
import numpy as np
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [4]:
# evaluation function
def perplexity(label, pred, ignore_label):
    label = label.T.reshape((-1,))
    loss = 0.
    for i in range(pred.shape[0]):
        if label[i] == ignore_label:
            break
        loss += -np.log(max(1e-10, pred[i][int(label[i])]))
    return np.exp(loss / label.size)

In [5]:
enc_train_input='data/1.en'
dec_train_input='data/1.ru'
num_buckets=1
num_layers=3
num_hidden=10
batch_size=1
iterations=1
expt_name='simple'
params_dir='params'
shuffle=False
reverse=True
top_words=0

In [ ]:


word2idx, idx2word = get_unified_vocab(enc_train_input, dec_train_input, top_words)
train_data_label = get_data_label(enc_train_input, dec_train_input, word2idx)

In [ ]:
train_iter = EncoderDecoderIter(train_data_label, word2idx, idx2word,
            num_hidden, num_layers, get_lstm_init_states, batch_size=batch_size,
            num_buckets=num_buckets, shuffle=shuffle, rev=reverse)

In [ ]:
def print_iter(iter):
    iter.reset()
    print('provide_data: ', iter.provide_data)
    print('provide_label: ', iter.provide_label)
    print('buckets: ', iter.buckets)
    print('buckets count: ', iter.buckets_count)
    print('assignments: ', iter.assignments)
    print('batch_size: ', iter.batch_size)
    for i, data_batch in enumerate(iter):
        print(i, data_batch.provide_data)
        print(i, data_batch.provide_label)
        print(i, data_batch.bucket_key)
        print(i, data_batch.data)
        for j, d in enumerate(data_batch.data):
            print(i, j, data_batch.data[j].shape)
            if j==0:
                print(data_batch.data[j].asnumpy())
        print(i, data_batch.label)
        print(i, data_batch.label[0].shape)
        print(i, data_batch.label[0].asnumpy())
#         print('\n')
        break

In [ ]:
print_iter(train_iter)

In [ ]:
context = mx.cpu()

model_args = {}
if os.path.isfile('%s/%s-symbol.json' % (params_dir, expt_name)):
    filelist = os.listdir(params_dir) # get list of params file
    paramfilelist = []
    for f in filelist:
        if f.startswith('%s-' % expt_name) and f.endswith('.params'):
            paramfilelist.append( int(re.split(r'[-.]', f)[1]) )
    last_iteration = max(paramfilelist)
    print('loading pretrained model %s/%s at epoch %d' % (params_dir, expt_name, last_iteration))
    tmp = mx.model.FeedForward.load('%s/%s' % (params_dir, expt_name), last_iteration)
    model_args.update({
        'arg_params' : tmp.arg_params,
        'aux_params' : tmp.aux_params,
        'begin_epoch' : tmp.begin_epoch
    })

num_labels = len(word2idx) - 1
model = mx.model.FeedForward(
    ctx           = context, # uses all the available CPU in the machine
    symbol        = get_lstm_sym_generator(num_layers, num_hidden, num_labels),
    num_epoch     = iterations,
    learning_rate = 0.01,
    momentum      = 0.0,
    wd            = 0.00001,
    initializer   = mx.init.Xavier(factor_type="in", magnitude=2.34),
    **model_args
)

if not os.path.exists(params_dir):
    os.makedirs(params_dir)

In [ ]:
model.fit(
    X = train_iter,
    eval_metric = mx.metric.np(perplexity, use_ignore=True, ignore_label=num_labels),
    batch_end_callback = [ mx.callback.Speedometer(batch_size, frequent=10) ],
    epoch_end_callback = [ mx.callback.do_checkpoint( '%s/%s' % (params_dir, expt_name) ) ]
)